In [ ]:
import numpy as np
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus, PULP_CBC_CMD


In [ ]:

# Define the dimensions of the matrix k
n = 5  # Number of columns (example value, adjust as needed)
m = 4  # Number of rows (example value, adjust as needed)
p = 0.5  # Probability value (example value, adjust as needed)

# Create the linear programming problem
prob = LpProblem("Maximize_Expression", LpMaximize)

# Create the binary matrix k
k = [[LpVariable(f'k_{i}_{j}', cat='Binary') for j in range(n)] for i in range(m)]

# Define the objective function
objective = lpSum(k[i][j] * p * lpSum(k[i_prime][j] * p + 0^k[i_prime][j] for i_prime in range(m)) for j in range(n) for i in range(m))
prob += objective

# Add the constraints
# 1. Sum of each row in matrix k must be greater than or equal to 1
for i in range(m):
    prob += lpSum(k[i][j] for j in range(n)) >= 1

# 2. Sum of each column in matrix k must be greater than or equal to 1
for j in range(n):
    prob += lpSum(k[i][j] for i in range(m)) >= 1

# Solve the problem
prob.solve(PULP_CBC_CMD())

# Print the status of the solution
print("Status:", LpStatus[prob.status])

# Print the values of the variables
for i in range(m):
    for j in range(n):
        print(f'k_{i}_{j} =', k[i][j].varValue)

# Print the value of the objective function
print("Objective value:", prob.objective.value())


In [ ]:
import numpy as np
from scipy.optimize import minimize

# Define the dimensions of the matrix k
n = 5  # Number of columns
m = 4  # Number of rows
p = 0.5  # Probability value

# Define the objective function
def objective(k_flat):
    k = k_flat.reshape((m, n))
    total_sum = 0
    for i in range(m):
        for j in range(n):
            prod_term = 1
            for i_prime in range(m):
                prod_term *= k[i_prime, j] * p + 0 ^ k[i_prime, j]
            total_sum += k[i, j] * p * prod_term
    print(total_sum)
    return -total_sum  # Minimize the negative to maximize the positive

# Define constraints
def row_sum_constraint(k_flat):
    k = k_flat.reshape((m, n))
    return np.sum(k, axis=1) - 1  # Each row sum must be >= 1

def col_sum_constraint(k_flat):
    k = k_flat.reshape((m, n))
    return np.sum(k, axis=0) - 1  # Each column sum must be >= 1

# Define bounds for the binary variables
bounds = [(0, 1) for _ in range(m * n)]

# Initial guess
initial_guess = np.random.rand(m, n)

# Define the constraints in scipy's format
constraints = [
    {'type': 'ineq', 'fun': row_sum_constraint},
    {'type': 'ineq', 'fun': col_sum_constraint}
]

# Run the optimization
result = minimize(objective, initial_guess.flatten(), method='SLSQP', bounds=bounds, constraints=constraints)

# Extract the result
k_optimal = result.x.reshape((m, n))

print("Optimal k matrix:")
print(k_optimal)
print("Objective value:", -result.fun)


In [ ]:
from pyomo.environ import *

# Define the model
model = ConcreteModel()

# Define the dimensions
n = 5
m = 4
p = 0.5

# Define sets
model.i = RangeSet(1, m)
model.j = RangeSet(1, n)

# Define binary variables
model.k = Var(model.i, model.j, domain=Binary)

# Define the objective function
def objective_function(model):
    return sum(model.k[i,j] * p * prod((model.k[i_prime,j] * p + (1 - model.k[i_prime,j])) for i_prime in model.i) for i in model.i for j in model.j)
model.objective = Objective(rule=objective_function, sense=maximize)

# Define constraints
def row_sum_constraint(model, i):
    return sum(model.k[i, j] for j in model.j) >= 1
model.row_sum_constraint = Constraint(model.i, rule=row_sum_constraint)

def col_sum_constraint(model, j):
    return sum(model.k[i, j] for i in model.i) >= 1
model.col_sum_constraint = Constraint(model.j, rule=col_sum_constraint)

# Solve the model
solver = SolverFactory('ipopt')
result = solver.solve(model, tee=True)

# Display the results
model.display()


In [ ]:
from pyomo.environ import *

# Define the model
model = ConcreteModel()

# Define the dimensions
n = 5
m = 4
p = 0.5
M = 1000  # Choose a suitable value for Big M

# Define sets
model.i = RangeSet(1, m)
model.j = RangeSet(1, n)

# Define binary variables
model.k = Var(model.i, model.j, domain=Binary)
model.y = Var(model.i, model.j, domain=Binary)

# Define auxiliary binary variables
model.M = Param(initialize=M)

# Define the objective function
def objective_function(model):
    return sum(model.k[i,j] * p * model.y[i,j] for i in model.i for j in model.j)
model.objective = Objective(rule=objective_function, sense=maximize)

# Define constraints
def linearization_constraint(model, i, j):
    return model.k[i,j] * p + (1 - model.k[i,j]) * model.M <= model.y[i,j] <= model.k[i,j] * p
model.linearization_constraint = Constraint(model.i, model.j, rule=linearization_constraint)

# Solve the model
solver = SolverFactory('glpk')
result = solver.solve(model, tee=True)

# Display the results
model.display()


In [ ]:
pip install pyomo
conda install -c conda-forge ipopt

In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus, PULP_CBC_CMD

# Define the dimensions
n = 5
m = 4
p = 0.5
M = 1000  # Big M value

# Create the linear programming problem
prob = LpProblem("Maximize_Objective", LpMaximize)

# Define binary variables
k = {(i, j): LpVariable(f'k_{i}_{j}', cat='Binary') for i in range(1, m+1) for j in range(1, n+1)}
y = {(i, j): LpVariable(f'y_{i}_{j}', cat='Binary') for i in range(1, m+1) for j in range(1, n+1)}
z = {(i, j): LpVariable(f'z_{i}_{j}', cat='Continuous', lowBound=0) for i in range(1, m+1) for j in range(1, n+1)}

# Define the objective function
prob += lpSum(z[i, j] for i in range(1, m+1) for j in range(1, n+1))

# Define linearization constraints
for i in range(1, m+1):
    for j in range(1, n+1):
        prob += z[i, j] >= k[i, j] * p + (1 - k[i, j]) * M
        prob += z[i, j] <= k[i, j] * p

# Solve the problem
prob.solve(PULP_CBC_CMD())

# Print the status of the solution
print("Status:", LpStatus[prob.status])

# Print the values of the variables
for i in range(1, m+1):
    for j in range(1, n+1):
        print(f'k_{i}_{j} =', k[i, j].varValue)
        print(f'y_{i}_{j} =', y[i, j].varValue)
        print(f'z_{i}_{j} =', z[i, j].varValue)

# Print the value of the objective function
print("Objective value:", prob.objective.value())


In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus, PULP_CBC_CMD

# Define the dimensions
n = 5
m = 4
p = 0.5
M = 1000  # Big M value

# Create the linear programming problem
prob = LpProblem("Maximize_Objective", LpMaximize)

# Define binary variables
k = {(i, j): LpVariable(f'k_{i}_{j}', cat='Binary') for i in range(1, m+1) for j in range(1, n+1)}
y = {(i, j): LpVariable(f'y_{i}_{j}', cat='Binary') for i in range(1, m+1) for j in range(1, n+1)}
z = {(i, j): LpVariable(f'z_{i}_{j}', cat='Continuous', lowBound=0) for i in range(1, m+1) for j in range(1, n+1)}

# Define the objective function
prob += lpSum(z[i, j] for i in range(1, m+1) for j in range(1, n+1))

# Define linearization constraints
for i in range(1, m+1):
    for j in range(1, n+1):
        prob += z[i, j] >= k[i, j] * p + (1 - k[i, j]) * M
        prob += z[i, j] <= k[i, j] * p

# Add constraints ensuring sum of each row and column >= 1
for i in range(1, m+1):
    prob += lpSum(k[i, j] for j in range(1, n+1)) >= 1

for j in range(1, n+1):
    prob += lpSum(k[i, j] for i in range(1, m+1)) >= 1

# Solve the problem
prob.solve(PULP_CBC_CMD())

# Print the status of the solution
print("Status:", LpStatus[prob.status])

# Print the values of the variables
for i in range(1, m+1):
    for j in range(1, n+1):
        print(f'k_{i}_{j} =', k[i, j].varValue)
        print(f'y_{i}_{j} =', y[i, j].varValue)
        print(f'z_{i}_{j} =', z[i, j].varValue)

# Print the value of the objective function
print("Objective value:", prob.objective.value())
print(k)


In [18]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus, PULP_CBC_CMD

# Define the dimensions
n_msg = 5
n_tag = 4
p = 0.5
M = 1000  # Big M value
P = [p**i for i in range(n_msg+1)]


# Create the linear programming problem
prob = LpProblem("Maximize_Objective", LpMaximize)

# Define binary variables
k = {(i, j): LpVariable(f'k_{i}_{j}', cat='Binary') for i in range(1, n_msg+1) for j in range(1, n_tag+1)}
z = {j: LpVariable(f'z_{j}', cat='Integer', lowBound=0) for j in range(1, n_tag+1)}
y = {(i,j,w): LpVariable(f'k_{i}_{j}_{w}', cat='Binary') for i in range(1, n_msg+1) for j in range(1, n_tag+1)for w in range(1, n_msg+1)}


# Define the objective function
prob += lpSum(k[i, j]*p*P[z[j]] for i in range(1, n_msg+1) for j in range(1, n_tag+1))

# Add constraints ensuring sum of each row and column >= 1
for i in range(1, n_msg+1):
    prob += lpSum(k[i, j] for j in range(1, n_tag+1)) >= 1

for j in range(1, n_tag+1):
    prob += lpSum(k[i, j] for i in range(1, n_msg+1)) >= 1

for j in range(1, n_tag+1):
    prob += z[j] == lpSum(k[i, j] for j in range(1, n_msg+1))

for i in range(1, n_msg+1):
    for j in range(1, n_tag+1):
            prob += z[j]-i >= M*y[i,j,i]
            prob += i-z[j] <= M*(1-y[i,j,i])

# Solve the problem
prob.solve(PULP_CBC_CMD())

# Print the status of the solution
print("Status:", LpStatus[prob.status])

# Print the values of the variables
for i in range(1, n_msg+1):
    for j in range(1, n_tag+1):
        print(f'k_{i}_{j} =', k[i, j].varValue)
        print(f'y_{i}_{j}_{i} =', y[i, j,w].varValue)
        print(f'z_{j} =', z[j].varValue)

# Print the value of the objective function
print("Objective value:", prob.objective.value())


[1.0, 0.5, 0.25, 0.125, 0.0625, 0.03125]


KeyError: (5, 5)